In [5]:
#Imports

import numpy as np
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report

##### Loading Training Data #########################

X=np.loadtxt('/home/subha/deepLearningCourse/homework1/train_data.txt',delimiter=',',usecols=(0,1))  #N*D
# train=train.T
# print(X) #N*D
y=np.loadtxt('/home/subha/deepLearningCourse/homework1/train_data.txt',delimiter=',',usecols=(2))
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
# print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
# print(onehot_encoded)
y=onehot_encoded


#Defining the derivative of sigmoid
def sigmoidDerivative(output):
    return output*(1-output)

#Yielding Batches
def next_batch(X,y,bS):
    for i in np.arange(0,X.shape[0],bS):
        yield(X[i:i+bS],y[i:i+bS])

totalloss=[]

def train():
    #Initializing dimensions
    input_dim=2
    hidden_dim=3
    output_dim=2

    #Hyper Parameters
    sigmoid= lambda x:1.0/(1.0+(np.exp(-x)))
    num_epochs=50
    learning_rate=.01
    batch_size = 1

    #Initializing weights
    W1= 2*np.random.random((input_dim,hidden_dim)) - 1
    W2= 2*np.random.random((hidden_dim,hidden_dim)) - 1
    W3= 2*np.random.random((hidden_dim,output_dim)) - 1

    print(W1.shape)
    for ep in range(num_epochs):
            iter_loss=[]
            num=np.random.randint(X.shape[0])
            batchX=X[num]
            batchY=y[num]
            print(batchX)
            
###########################Forward Propogation ##########################################################################
            z1=np.dot(batchX,W1)
            a1=sigmoid(z1)
            z2=np.dot(a1,W2)
            a2=sigmoid(z2)        
            z3=np.dot(a2,W3)
            #softmax scores
            exp_scores = np.exp(z3)
            probs=exp_scores/np.sum(exp_scores)
            print('probs',probs)
            l2_loss=np.sum(abs(probs-batchY)**2)
            

################################ Backward propogation ###################################################################
            dl_dy=2*((probs-batchY))
            dscores=np.matrix([[(probs[0]*probs[1]), 0], [0, (probs[0]*probs[1])]])
            layer3_delta=dl_dy*dscores
            layer2_error=layer3_delta.dot(W3.T)
            layer2_delta=np.multiply(layer2_error,(sigmoidDerivative(z2)))
            layer1_error=layer2_delta.dot(W2.T)
            layer1_delta=np.multiply(layer1_error,(sigmoidDerivative(z1)))
            a2=np.matrix(a2)
            batchX=np.matrix(batchX)
            dw3=a2.T.dot(layer3_delta)
            dw2=a1.T.dot(layer2_delta.T)
            dw1=batchX.T.dot(layer1_delta)
            W1-=learning_rate*dw1
            W2-=learning_rate*dw2
            W3-=learning_rate*dw3
            print("L2 loss: {0}".format(l2_loss))
    model={}
    model['W1']=W1
    model['W2']=W2
    model['W3']=W3
    print("weights{0}".format(model))
    return model
def predict(model, x):
        sigmoid= lambda x:1.0/(1.0+(np.exp(-x)))
        W1, W2, W3= model['W1'], model['W2'], model['W3']
        # Forward propagation
        z1=np.dot(x,W1)
        a1=sigmoid(z1)
        z2=np.dot(a1,W2)
        a2=sigmoid(z2)        
        z3=np.dot(a2,W3)
        #softmax scores
        exp_scores = np.exp(z3)
        probs=exp_scores/np.sum(exp_scores)
        print(probs.shape)
        return np.argmax(probs, axis=1)

**Forward Propogation**


$Z1=W1.X1$
<br>

$a1 = sigmoid(Z1)$
<br>

$Z2=W2.a1$
<br>

$a2 = sigmoid(Z2)$
<br>

$Z3 = W3.a2$
<br>

$y1= e^{Z_1^3}/(e^{Z_1^3}+e^{Z_2^3})$ , $y2= e^{Z_2^3}/(e^{Z_1^3}+e^{Z_2^3})$
<br>

$L=| (y1 - \hat{y})^2 | + | (y2 - \hat{y})^2 |$
<br>



**Backward Propogation**
<br>
$1. \frac{\partial L }{\partial W3} = \frac{\partial L }{\partial y} * \frac{\partial y }{\partial Z3} *\frac{\partial Z3 }{\partial W3} $  
<br>

$\frac{\partial L }{\partial y} = 2 *((y1 - \hat{y}) + (y2 - \hat{y}))$
<br>

$\frac{\partial y }{\partial Z3} (Derivative of Softmax) = \left(\begin{array}{cc} 0 & y1*y2\\y1*y2 & 0\end{array}\right)$
<br>

$ Let \ \delta3 = \frac{\partial L }{\partial y} * \frac{\partial y }{\partial Z3}$
<br>


$2. \frac{\partial L }{\partial W2} =  \delta3  * \frac{\partial Z3}{\partial a2} * \frac{\partial a2 }{\partial Z2} *\frac{\partial Z2 }{\partial W2} $
<br>

$ \frac{\partial Z3}{\partial a2} = W3. T$
<br>

$ \frac{\partial a2 }{\partial Z2} = \sigma '(Z2)$
<br>

$\frac{\partial Z2 }{\partial W2}=a1.T$
<br>

$ Let \ \delta2 = \delta3  * \frac{\partial Z3}{\partial a2} * \frac{\partial a2 }{\partial Z2}$
<br>


$3. \frac{\partial L }{\partial W1} =  \delta2  * \frac{\partial Z2}{\partial a1} * \frac{\partial a1 }{\partial Z1} 
*\frac{\partial Z1 }{\partial W1} $
<br>

$ \frac{\partial Z2}{\partial a1} = W2. T$
<br>

$\frac{\partial a1 }{\partial Z1}=\sigma '(Z1)$
<br>

$\frac{\partial Z1 }{\partial W1} = X.T$

<br>
T - Transpose

In [6]:
######################################## Testing #######################################################################   
Xt=np.loadtxt('/home/subha/deepLearningCourse/homework1/test_data.txt',delimiter=',',usecols=(0,1))  #N*D
yt=np.loadtxt('/home/subha/deepLearningCourse/homework1/test_data.txt',delimiter=',',usecols=(2))
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(yt)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
yt=onehot_encoded
model=train()  
labels=predict(model,Xt)
tn=['0','1']
print(classification_report(integer_encoded,labels,tn))
# print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

(2, 3)
[-1.80659  2.31651]
('probs', array([ 0.43385081,  0.56614919]))
L2 loss: 0.376453043425
[-0.62594  1.86727]
('probs', array([ 0.43340356,  0.56659644]))
L2 loss: 0.375677299166
[ 0.65602 -2.56565]
('probs', array([ 0.4816063,  0.5183937]))
L2 loss: 0.537464050985
[ 2.00991  0.08473]
('probs', array([ 0.46083023,  0.53916977]))
L2 loss: 0.581408075185
[-1.34109  1.12484]
('probs', array([ 0.43901456,  0.56098544]))
L2 loss: 0.385467574252
[-2.93278  2.32369]
('probs', array([ 0.43474058,  0.56525942]))
L2 loss: 0.377998747593
[ 2.83315 -3.45788]
('probs', array([ 0.47996189,  0.52003811]))
L2 loss: 0.540879270005
[-1.47049  1.117  ]
('probs', array([ 0.43733754,  0.56266246]))
L2 loss: 0.382528253446
[-2.03559  2.38716]
('probs', array([ 0.43183556,  0.56816444]))
L2 loss: 0.372963899247
[ 2.48954 -0.68022]
('probs', array([ 0.46845868,  0.53154132]))
L2 loss: 0.565072357761
[-1.78767  2.71927]
('probs', array([ 0.43031837,  0.56968163]))
L2 loss: 0.370347801593
[ 2.89598 -4.116

/home/subha/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:44: RuntimeWarning: overflow encountered in exp
/home/subha/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:28: RuntimeWarning: overflow encountered in multiply
/home/subha/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:88: RuntimeWarning: invalid value encountered in subtract
/home/subha/anaconda3/envs/python2/lib/python2.7/site-packages/numpy/lib/arraysetops.py:459: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)
/home/subha/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [7]:
#################################Solution Using Framework ############################################################

from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from sklearn.metrics import classification_report
from keras.layers import Dropout
import numpy as np
model=Sequential()
model.add(Dense(3,input_dim=2,activation='sigmoid' ))
model.add(Dropout(0.2))
model.add(Dense(3,activation='sigmoid' ))
model.add(Dropout(0.2))
model.add(Dense(2,activation='softmax' ))
X=np.loadtxt('/home/subha/deepLearningCourse/homework1/train_data.txt',delimiter=',',usecols=(0,1))
# train=train.T
# print(X) #N*D


y=np.loadtxt('/home/subha/deepLearningCourse/homework1/train_data.txt',delimiter=',',usecols=(2))
y=to_categorical(y,2)
# y=[[i] for i in y]
# y=np.array(y)
Xtest=np.loadtxt('/home/subha/deepLearningCourse/homework1/test_data.txt',delimiter=',',usecols=(0,1))

# train=train.T
# print(X) #N*D


ytest=np.loadtxt('/home/subha/deepLearningCourse/homework1/test_data.txt',delimiter=',',usecols=(2),dtype='S32')
ytemp=ytest
ytest=to_categorical(ytest,2)
# ytest=[[i] for i in ytest]
# ytest=np.array(ytest)
model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['accuracy'])
# Fit the model
model.fit(X, y, epochs=50, batch_size=1)
# evaluate the model
# target
scores = model.evaluate(Xtest, ytest)
probabilities = model.predict(Xtest)
labels=np.argmax(probabilities,axis=1)
labels=np.array(labels)
labels=labels.astype(int)
ytemp=ytemp.astype(int)

print(labels)
tn=['0','1']
print(classification_report(ytemp,labels,tn))
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Using TensorFlow backend.


Epoch 1/50
1000/1000 [==============================] - 16s - loss: 0.2398 - acc: 0.6360        

Precision, Recall and f1-score Calculation

TP - Number of time correctly identified 1 when it was 1
FN - Number of time identified 0 when it was 1
FP - Number of times identified 1 when it was 0

Recall = TP/ TP + FN
Precision = TP/ TP+FN

$F1 Score= Precision*Recall/(Precision + Recall)$